In [1]:
import os

In [3]:
%pwd
os.chdir("../")
%pwd

'd:\\Visual Studio 2019\\MLops\\Chest Cancer Detection suing MLFlow and DVC'

In [24]:
# UPDATE ENTITY

from dataclasses import dataclass
from pathlib import Path

# defineing entity
# this entity defines the return type of a function
# this is the return tupe of the data_ingestion configuration
@dataclass(frozen=True) # frozen=True means we will not allow any more functionality to be added , ie if the data_ingestion returns one more value or one less , it throws error
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
# when we create the data_ingestion configuration this entity will tell what that configuration will return 

# @dataclass ,is a decorator that will enable us to write a class without using self

In [25]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.cnnClassifier.utils.common import read_yaml  , create_directories


In [26]:
#  UPDATE THE CONFIGURATION MANAGER IN SRC CONFIG

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [27]:
import os
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-05 19:48:34,281: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 19:48:34,283: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 19:48:34,284: INFO: common: created directory at: artifacts]
[2023-12-05 19:48:34,286: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-05 19:48:34,288: INFO: 1545650104: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=6d8db9f1-bb8d-4533-83dc-7916c699c602
To: d:\Visual Studio 2019\MLops\Chest Cancer Detection suing MLFlow and DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:12<00:00, 3.80MB/s]

[2023-12-05 19:48:48,972: INFO: 1545650104: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
